In [ ]:
def add_variables(n):
     n.model.add_variables(0, 1000, coords=[n.snapshots,n_obj_generators_committable_reserve], name="p_prime")          # p'           Power generated above minimum by generator g at time t (MW), ≥ 0.
     n.model.add_variables(0, 1000, coords=[n.snapshots,n_obj_generators_committable_reserve], name="p_bar_prime")      # p.bar.prime  Maximum power available above minimum from generator g at time t (MW), ≥0
     n.model.add_variables(0, 5000, coords=[n.snapshots,n_obj_generators_committable_reserve], name="r_up_com")         # r            Spinning reserves provided by generator g at time t (MW), ≥ 0.
     n.model.add_variables(0, 1000, coords=[n.snapshots,n_obj_generators_committable_reserve], name="p_bar")  
     n.model.add_variables(0, 1000, coords=[n.snapshots,n_obj_generators_bH], name="r_up_bH")         # reserve provision by big hydro
     n.model.add_variables(0, 1000, coords=[n.snapshots,n_obj_generators_bH], name="r_down_bH")       # down reserve by big hydro
     n.model.add_variables(0, 10000, coords=[n.snapshots,n_obj_generators_bH], name="bh_out_flow") 
      # Add variables
     # https://optimization-online.org/wp-content/uploads/2018/11/6930.pdf      
     print("variable added")
     return(n)
     
def reserve_constraints(n):
         
     # Reserve provision constraints for committable generators

     def eq_12(n):
          print(12)
          # equation 12 in https://optimization-online.org/wp-content/uploads/2018/11/6930.pdf 
          lhs = n.model["p_prime"].loc[:,n_obj_generators_committable_reserve] +   g_status.loc[:,n_obj_generators_committable_reserve] * n.generators_t["p_min_pu"][n_obj_generators_committable_reserve] * n.generators["p_nom"].loc[n_obj_generators_committable_reserve] - n.model["Generator-p"].loc[:,n_obj_generators_committable_reserve]
          n.model.add_constraints(lhs  == 0,name = "eq_12")
          print("eq12")

     def eq_13(n):
          # equation 13 in https://optimization-online.org/wp-content/uploads/2018/11/6930.pdf 
          lhs = n.model["p_bar_prime"].loc[:,n_obj_generators_committable_reserve] + g_status.loc[:,n_obj_generators_committable_reserve] * n.generators_t["p_min_pu"][n_obj_generators_committable_reserve] * n.generators["p_nom"].loc[n_obj_generators_committable_reserve] - n.model["p_bar"].loc[:,n_obj_generators_committable_reserve]
          n.model.add_constraints(lhs  == 0,name = "eq_13")
          print("eq13")

     def eq_14(n):
          # equation 14 in https://optimization-online.org/wp-content/uploads/2018/11/6930.pdf 
          lhs = n.model["p_prime"].loc[:,n_obj_generators_committable_reserve] + n.model["r_up_com"].loc[:,n_obj_generators_committable_reserve] - n.model["p_bar_prime"].loc[:,n_obj_generators_committable_reserve]
          n.model.add_constraints(lhs  == 0,name = "eq_14")
          print("eq14")
     def eq_15(n):
          # equation 15 in https://optimization-online.org/wp-content/uploads/2018/11/6930.pdf 
          lhs = n.model["Generator-p"].loc[:,n_obj_generators_committable_reserve]  +  n.model["r_up_com"].loc[:,n_obj_generators_committable_reserve]  - n.model["p_bar"].loc[:,n_obj_generators_committable_reserve]
          n.model.add_constraints(lhs  == 0,name = "eq_15")
          print("eq15")     
     def eq_16(n):
          # equation 16 in https://optimization-online.org/wp-content/uploads/2018/11/6930.pdf 
          lhs = n.model["Generator-p"].loc[:,n_obj_generators_committable_reserve]  
          rhs =  n.model["p_bar"].loc[:,n_obj_generators_committable_reserve]
          n.model.add_constraints(lhs  <= rhs,name = "eq_16")
          print("eq16")
     def eq_17(n):
          # equation 17 in https://optimization-online.org/wp-content/uploads/2018/11/6930.pdf 
          lhs = n.model["p_prime"].loc[:,n_obj_generators_committable_reserve] 
          rhs =  n.model["p_bar_prime"].loc[:,n_obj_generators_committable_reserve]
          n.model.add_constraints(lhs  <= rhs,name = "eq_17")     
          print("eq17")
     def eq_22(n):   # check whether needed
          # equation 23A in https://optimization-online.org/wp-content/uploads/2018/11/6930.pdf 
          rhs3 = 3
          sns  =n.snapshots
          lns = len(n.snapshots)
          sns = sns.drop(n.snapshots[lns-1])
     
          tmp = 0
        
          for t in sns:
               t_p1 = n.snapshots[tmp+1]
               
               lhs = n.model["p_bar"].loc[t,n_obj_generators_committable_reserve] 
               rhs1 = g_status.loc[t,n_obj_generators_committable_reserve] * n.generators["p_nom"][n_obj_generators_committable_reserve] 
               rhs2 = g_start_up.loc[t,n_obj_generators_committable_reserve] * (n.generators["p_nom"][n_obj_generators_committable_reserve] -n.generators["ramp_limit_start_up"][n_obj_generators_committable_reserve])
               rhs3 = g_shut_down.loc[t_p1,n_obj_generators_committable_reserve] *(n.generators["p_nom"][n_obj_generators_committable_reserve] -n.generators["ramp_limit_start_up"][n_obj_generators_committable_reserve] )
               tmp = tmp + 1
               n.model.add_constraints(lhs  <= rhs1 - rhs2 - rhs3 ,name = "eq_22A"+str(tmp))

     # Up_Reserve provision constraints for big hydro
          
     def up_reserve_provision_by_bH(n): 
          # big hydro up-reserve provision
          lhs =  n.model["r_up_bH"].loc[:,n_obj_generators_bH]  + n.model["Generator-p"].loc[:,n_obj_generators_bH]
          rhs =  n.generators["p_nom"].loc[n_obj_generators_bH] #n.generators_t["p_max_pu"].loc[:,n_obj_generators_bH]
          n.model.add_constraints(lhs  <= rhs,name = "up_reserve_provision_by_bH")

     def agg_up_reserve_provision_by_bH(n): 
     #     # big hydro up-reserve provision
          lhs = n.model["Generator-p"].loc[:,n_obj_generators_bH] + n.model["r_up_bH"].loc[:,n_obj_generators_bH]  
          rhs = n.generators["p_nom"].loc[n_obj_generators_bH] #n.generators_t["p_max_pu"].loc[:,n_obj_generators_bH]
          n.model.add_constraints(lhs  <= rhs,name = "up_reserve_provision_by_bH")

     # Down_Reserve provision constraints for big hydro
     def down_reserve_provision_by_bH(n): 
          # big hydro down-reserve provision
          lhs = n.model["Generator-p"].loc[:,n_obj_generators_bH]  - n.model["r_down_bH"].loc[:,n_obj_generators_bH] 
          n.model.add_constraints(lhs  >= 0,name = "down_reserve_provision_by_bH")

     ########## RESERVE CONSTRAINTS     

     def aggregated_reserve_up_constraint(n):
          # aggregated up-reserve provision is greater than or equal the down_reserve_requirement
          rhs = up_reserve_requirement
          if allow_bh_reserve_prvision == "yes":
               lhs =  n.model["r_up_bH"].loc[:,n_obj_generators_bH].sum(dims = ["Generator"])+ n.model["r_up_com"].loc[:,n_obj_generators_committable_reserve].sum(dims = ["Generator"])  
               
          else:
               
               lhs = n.model["r_up_com"].loc[:,n_obj_generators_committable_reserve].sum(dims = ["Generator"])   
          n.model.add_constraints(lhs  >= rhs,name = "aggregated_reserve_up_constraint")

     def aggregated_reserve_down_constraint(n):
          # aggregated down-reserve provision is greater than or equal the down_reserve_requirement
          rhs =  down_reserve_requirement
          if allow_bh_reserve_prvision == "yes":
               lhs =  n.model["p_prime"].loc[:,n_obj_generators_committable_reserve].sum(dims = ["Generator"]) + n.model["r_down_bH"].loc[:,n_obj_generators_bH].sum(dims = ["Generator"])
          else: 
               lhs =  n.model["p_prime"].loc[:,n_obj_generators_committable_reserve].sum(dims = ["Generator"])   
          n.model.add_constraints(lhs  >= rhs ,name = "aggregated_reserve_down_constraint")


       
     g_status = n.model["Generator-status"].rename({"Generator-com": "Generator"}) 
     g_start_up = n.model["Generator-start_up"].rename({"Generator-com": "Generator"}) 
     g_shut_down = n.model["Generator-shut_down"].rename({"Generator-com": "Generator"}) 

    
     eq_12(n)
     eq_13(n)
     eq_14(n)
     eq_15(n)
     eq_16(n)
     eq_17(n)
     eq_22(n)
     up_reserve_provision_by_bH(n)   
     down_reserve_provision_by_bH(n)

     aggregated_reserve_up_constraint(n)
     aggregated_reserve_down_constraint(n)

     return(n)
